## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd
import pickle as pkl

In [2]:
df=pd.read_csv('dataset/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


<ul>
 <li>1: unreliable</li>
 <li>0: reliable</li>
</ul>

In [4]:
###Drop Nan Values
df=df.dropna()

In [5]:
## Get the Independent Features
X=df.drop('label',axis=1)

In [6]:
## Get the Dependent features
y=df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.1.0'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [13]:
messages=X.copy()

In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.reset_index(inplace=True)

In [16]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

## Apply Stemming on text data
<b> Stemming takes long time on large data</b>

In [19]:
try:
    ## Try to load saved corpus
    with open('corpus.pkl', 'rb') as f:
        corpus = pkl.load(f)
except:
    ## If not saved before create a corpus
    
    ### Dataset Preprocessing
    from nltk.stem.porter import PorterStemmer
    ps = PorterStemmer()
    corpus = []
    for i in range(0, len(messages)):
        review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
        review = review.lower()
        review = review.split()

        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)

    with open('corpus.pkl', 'wb') as f:
        pkl.dump(corpus, f)

In [20]:
corpus[:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [21]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[168, 4401, 3047, 514, 1491, 2082, 4057, 4547, 2848, 189],
 [703, 3833, 2542, 145, 1168, 1460, 3147],
 [4673, 3426, 1313, 3994],
 [4778, 4248, 794, 1098, 503, 561],
 [3817, 1168, 3011, 3893, 3177, 1843, 1168, 2629, 1443, 1314],
 [522,
  637,
  3251,
  3650,
  973,
  1152,
  2029,
  2145,
  1499,
  858,
  102,
  147,
  1661,
  2286,
  3147],
 [176, 1875, 1240, 95, 3865, 1215, 4645, 4170, 2501, 1786, 4146],
 [4816, 2725, 3271, 3221, 4925, 1374, 1152, 4624, 2501, 1786, 4146],
 [3038, 872, 4764, 1160, 2721, 1576, 717, 3681, 1152, 4144],
 [2143, 3070, 2472, 2484, 4024, 1577, 3749, 1102],
 [2688, 1506, 1993, 922, 2686, 3766, 1674, 2287, 4681, 1229, 488],
 [1098, 2687, 1491, 1576, 1152, 4925],
 [2474, 354, 2177, 673, 1078, 191, 475, 2016, 3919],
 [3980, 1591, 4555, 4176, 2169, 4875, 3763, 2501, 1786, 4146],
 [106, 3710, 3867, 3872, 4492, 2501, 1786, 4146],
 [3857, 38, 708, 3883, 1895, 2973, 1576, 2264, 4565, 1241],
 [1727, 3672, 3833],
 [2771, 2824, 421, 904, 1152, 3150, 2794, 3147],
 [3151,

### Embedding Representation

In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4547 2848  189]
 [   0    0    0 ... 1168 1460 3147]
 [   0    0    0 ... 3426 1313 3994]
 ...
 [   0    0    0 ... 2501 1786 4146]
 [   0    0    0 ... 1520 3456  170]
 [   0    0    0 ... 1479 2554 1882]]


In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  168,
       4401, 3047,  514, 1491, 2082, 4057, 4547, 2848,  189])

## Model Building

In [24]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
len(embedded_docs),y.shape

(18285, (18285,))

In [26]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [27]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [29]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 15s 1ms/sample - loss: 0.3329 - accuracy: 0.8424 - val_loss: 0.1988 - val_accuracy: 0.9145
Epoch 2/10
12250/12250 [==============================] - 5s 418us/sample - loss: 0.1435 - accuracy: 0.9438 - val_loss: 0.2050 - val_accuracy: 0.9201
Epoch 3/10
12250/12250 [==============================] - 4s 356us/sample - loss: 0.0980 - accuracy: 0.9635 - val_loss: 0.2179 - val_accuracy: 0.9178
Epoch 4/10
12250/12250 [==============================] - 5s 416us/sample - loss: 0.0662 - accuracy: 0.9761 - val_loss: 0.2541 - val_accuracy: 0.9112
Epoch 5/10
12250/12250 [==============================] - 5s 385us/sample - loss: 0.0445 - accuracy: 0.9857 - val_loss: 0.3133 - val_accuracy: 0.9178
Epoch 6/10
12250/12250 [==============================] - 4s 333us/sample - loss: 0.0275 - accuracy: 0.9922 - val_loss: 0.4021 - val_accuracy: 0.9153
Epoch 7/10
12250/12250 [============================

### Adding Dropout 

In [30]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model_new=Sequential()
model_new.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model_new.add(Dropout(0.3))
model_new.add(LSTM(100))
model_new.add(Dropout(0.3))
model_new.add(Dense(1,activation='sigmoid'))
model_new.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_new.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 11s 867us/sample - loss: 0.3467 - accuracy: 0.8320 - val_loss: 0.2024 - val_accuracy: 0.9130
Epoch 2/10
12250/12250 [==============================] - 5s 430us/sample - loss: 0.1556 - accuracy: 0.9393 - val_loss: 0.1956 - val_accuracy: 0.9186
Epoch 3/10
12250/12250 [==============================] - 5s 407us/sample - loss: 0.1075 - accuracy: 0.9602 - val_loss: 0.2127 - val_accuracy: 0.9125
Epoch 4/10
12250/12250 [==============================] - 5s 444us/sample - loss: 0.0825 - accuracy: 0.9722 - val_loss: 0.2354 - val_accuracy: 0.9157
Epoch 5/10
12250/12250 [==============================] - 6s 452us/sample - loss: 0.0606 - accuracy: 0.9793 - val_loss: 0.2717 - val_accuracy: 0.9137
Epoch 6/10
12250/12250 [==============================] - 6s 457us/sample - loss: 0.0435 - accuracy: 0.9840 - val_loss: 0.3145 - val_accuracy: 0.9130
Epoch 7/10
12250/12250 [==========================

### Performance Metrics And Accuracy

In [31]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [32]:
def model_Validate(local_model):
    y_pred=local_model.predict_classes(X_test)
    print('Confusion matrix: \n', confusion_matrix(y_test,y_pred))
    print('Accuracy: ',accuracy_score(y_test,y_pred))

In [33]:
model_Validate(model)

Confusion matrix: 
 [[3155  264]
 [ 284 2332]]
Accuracy:  0.9091963545981773


In [34]:
model_Validate(model_new)

Confusion matrix: 
 [[3086  333]
 [ 185 2431]]
Accuracy:  0.9141673570836786
